In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dgeison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


spark = SparkSession.builder\
    .master('local[*]')\
    .appName('WordCloud')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/12/09 00:36:02 WARN Utils: Your hostname, pc_branco resolves to a loopback address: 127.0.1.1; using 172.17.188.41 instead (on interface eth0)
22/12/09 00:36:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 00:36:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
def trata_tweets(df):
    words = df\
        .select(f.explode(f.split(f.lower('_c0'),' ')).alias('word')) \
        .withColumn('word',f.regexp_replace('word', r'htpp\S+','')) \
        .withColumn('word',f.regexp_replace('word', r'@\w+', '')) \
        .withColumn('word',f.regexp_replace('word', 'rt', '')) \
        .na.replace('', None) \
        .na.drop()
    return words

In [7]:
stops = stopwords.words('portuguese')
stops.append('copa')
plt.figure(figsize=(20,10))

while True:
    try:
        words = spark.read.csv('./csv', encoding='utf-8')
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + ' ' + row['word']
            
        wordcloud = WordCloud(
            stopwords=stops, 
            background='black',
            width=1920,
            height=1080,
            max_words=100            
        ).generate(all_words)
        
        plt.cla()
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break

AnalysisException: Path does not exist: file:/home/dgeison/spark_streaming_de_dados/csv

<Figure size 2000x1000 with 0 Axes>